In [1]:
import os
from xml.etree import ElementTree as ET
from datetime import datetime

# Almacenamieto de datos

In [2]:
METADATA_FIELDS = [
    ("identificador", None),
    ("titulo", None),
    ("departamento", None),
    ("fecha_publicacion", lambda x: datetime.strptime(x, "%Y%m%d")),
    ("origen_legislativo", None),
    ("rango", None),
]

def jsonify_boe_entry(xml):
    entry_json = {}

    # get metadata
    metadata = xml.find("metadatos")
    for tag, parser in METADATA_FIELDS:
        element = metadata.find(tag)
        if element is None: continue
        text = element.text
        entry_json[tag] = parser(text) if parser is not None else text
    
    # get topics
    entry_json["materias"] = [topic.text for topic in xml.findall(".//materia")]

    # get references
    past_refs = []
    for ref in xml.findall(".//anterior"):
        past_refs.append({
            "identificador": ref.get("referencia"),
            "texto": ref.find("texto").text,
        })
    entry_json["anteriores"] = past_refs

    future_refs = []
    for ref in xml.findall(".//posterior"):
        future_refs.append({
            "identificador": ref.get("referencia"),
            "texto": ref.find("texto").text,
        })
    entry_json["posteriores"] = future_refs

    # get XML text
    xml_text = xml.find("texto")
    html_text = ET.tostring(xml_text, encoding='utf8', method="html").decode('utf8')
    html_text = "\n".join(html_text.split("\n")[1:-1])
    entry_json["texto"] = html_text

    # get paragraphs
    entry_json["parrafos"] = [paragraph.text for paragraph in xml_text.findall(".//p")]

    return entry_json

In [3]:
# read all files in downloads folder, recursively
def read_xml_files(path):
    for r, _, f in os.walk(path):
        for file in f:
            if '.xml' not in file or "BOE" not in file:
                continue
            yield os.path.join(r, file)

In [4]:
def process_data():
    data = {}
    i = 0
    #itero por las carpetas de boe
    for anio in os.listdir('boe/dias'):
        #itero por los archivos de cada carpeta
        for mes in os.listdir(f'boe/dias/{anio}'):
            #si el archivo es un pdf
            for dia in os.listdir(f'boe/dias/{anio}/{mes}'):
                path_xmls = os.path.join('boe', 'dias', anio, mes, dia, 'xmls')
                print(path_xmls)
                for xml in os.listdir(path_xmls):
                    path_xml = os.path.join(path_xmls, xml)
                    xml = ET.parse(path_xml)
                    entry_json = jsonify_boe_entry(xml)
                    data[entry_json['identificador']] = entry_json

    return data

In [5]:
def get_embeddings(input_text, model, tokenizer, max_length=512):
    enc = tokenizer(input_text, return_tensors='pt', truncation=True, max_length=max_length)
    output = model.encoder(
        input_ids=enc['input_ids'],
        attention_mask=enc['attention_mask'],
        return_dict=True
        )
    return output.last_hidden_state.tolist()[0][0]

## ELASTICSEARCH

In [6]:
#!pip install elasticsearch sentencepiece

In [7]:
from elasticsearch import Elasticsearch
import json

#client = Elasticsearch("http://elasticsearch:9200")
client = Elasticsearch("http://localhost:9200")

### Crear index

In [8]:
config = {
    "mappings": {
        "properties": {
            "texto": {
                "type": "text"
            },
            "semantic_embeddig": {
                "type": "dense_vector",
                "dims": 512,
                "index": True,
                "similarity": "cosine"
            },
            #"titulo": {"type": "text"},
            "doc_id": {"type": "text"},
            #"fecha_publicacion": {"type": "date"}
        }
    },
    "settings": {
        "analysis": {
            "analyzer": {
                "my_analyzer": {
                "tokenizer": "keyword",
                "char_filter": [
                    "html_strip"
                ]
                }
            }
        },
        "number_of_shards": 2,
        "number_of_replicas": 1
    }
}

In [9]:
import torch
from transformers import T5Model, T5Tokenizer

model = T5Model.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

d:\Proyectos\venv310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
client = Elasticsearch("http://localhost:9200")
my_index = "boe"
try:
    client.indices.delete(index=my_index)
    print("Index deleted")
except:
    print("Index does not exist")
try:
    client.indices.create(
        index = my_index,
        settings = config["settings"],
        mappings = config["mappings"]
    )
except Exception as error:
    print("Error:", error)

Index deleted


### Poblar índices

In [11]:
data = process_data()
#data[list(data.keys())[-5]]

boe\dias\2023\10\25\xmls
boe\dias\2023\10\26\xmls
boe\dias\2023\10\27\xmls
boe\dias\2023\10\28\xmls
boe\dias\2023\10\30\xmls
boe\dias\2023\10\31\xmls
boe\dias\2023\11\01\xmls
boe\dias\2023\11\02\xmls
boe\dias\2023\11\03\xmls
boe\dias\2023\11\04\xmls
boe\dias\2023\11\06\xmls


In [12]:
for key in list(data.keys())[:1]:
    print(key)
    fecha = data[key]['fecha_publicacion']
    doc_id = key
    full_text = data[key]['texto']
    titulo = data[key]['titulo']
    semantic_embeddigs = []
    
    for i, text in enumerate(data[key]['parrafos']):
        semantic_embeddigs = get_embeddings(text, model, tokenizer)
        document = {
                "doc_id": doc_id,
                "embeddings": semantic_embeddigs,
                "texto": text,
                #"fecha_publicacion": fecha,
                #"full_text": full_text,
                #"titulo": titulo
            }

        try:
            client.index(
                index = my_index,
                document = document
            )
        except Exception as e:
            print(e)
            print(f"len: {len(document)}")
            print(fecha)
            print(id)
            print(document)
            break

BOE-A-2023-21844


In [13]:
my_query = {
    "match_all": {}
}

res = client.search(index=my_index, body={"query": my_query}, size=100)
print(f"Numero de entradas: {len(res['hits']['hits'])}")
print(f"Entradas encontradas: {res['hits']['hits'][0]['_source'].keys()}")
print(f"Nombre documento: {res['hits']['hits'][0]['_source']['doc_id']}")
print(f"Tamaño embeddings: {len(res['hits']['hits'][0]['_source']['embeddings'])}")
print(f"Texto: {res['hits']['hits'][0]['_source']['texto']}")

Numero de entradas: 100
Entradas encontradas: dict_keys(['doc_id', 'embeddings', 'texto'])
Nombre documento: BOE-A-2023-21844
Tamaño embeddings: 512
Texto: El Real Decreto 526/2014, de 20 de junio, por el que se establece la lista de las enfermedades de los animales de declaración obligatoria y se regula su notificación, da cumplimiento a lo dispuesto en la Directiva 82/894/CEE del Consejo, de 21 de diciembre de 1982, relativa a la notificación de las enfermedades de los animales en la Comunidad, así como a las obligaciones que el Reino de España tiene como país miembro de la Organización Mundial de Sanidad Animal (OMSA).


### Búsqueda léxica

In [14]:
string_to_search = "seguridad jurídica"
lexic_query = {
    "fields": ["texto"],
    "query" : string_to_search,
}
my_query = {
    "simple_query_string":{
        "fields": ["texto"],
        "query": string_to_search
    }
}
res = client.search(index=my_index, query=my_query)
for resp in res['hits']['hits']:
    print(resp['_source']['doc_id'])
    print(resp['_source']['texto'])


BOE-A-2023-21844
Razones de seguridad jurídica, dado el alcance de las modificaciones a introducir en el Real Decreto 526/2014, de 20 de junio, aconsejan la aprobación de un nuevo real decreto, aunque la presente norma mantenga los elementos esenciales en cuanto a estructura y contenido del precedente, adecuándose al nuevo marco normativo antes expuesto.
BOE-A-2023-21844
Este proyecto se adecúa a los principios de buena regulación a que se refiere el artículo 129 de la Ley 39/2015, de 1 de octubre, de Procedimiento Administrativo Común de las Administraciones Públicas. En concreto, cumple con los principios de necesidad y eficacia, pues se articulan los instrumentos necesarios para la notificación de las enfermedades de declaración obligatoria, siendo una norma de carácter básico con rango de real decreto que deroga la normativa en vigor con el fin de adaptar el mecanismo de notificación a lo dispuesto en la normativa europea, por lo que constituye el instrumento más eficaz para cumpli

### Búsqueda semántica

In [15]:
string_to_search = "Declaración  de las enfermedades de los animales"
parametrs = {
    "field": "semantic_embeddig",
    "query_vector" : get_embeddings(string_to_search, model, tokenizer),
    "k": 2,
    "num_candidates": 5
}
res = client.search(index=my_index, knn=parametrs)
print("Respuestas búsqueda semántica:")
for resp in res['hits']['hits']:
    print(resp['_source']['doc_id'])
    print(resp['_source']['texto'])

Respuestas búsqueda semántica:
